In [1]:
import tensorflow

from tensorflow. keras import layers
from tensorflow. keras import Model
from tensorflow. keras. models import Sequential
from tensorflow. keras.applications.vgg16 import VGG16

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
ImageDataGenerator??

In [3]:
train_augmentation = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
  )

In [4]:
train_generator = train_augmentation.flow_from_directory(
    directory='./Training',
    target_size=(128,128),
    batch_size=128,
    class_mode='binary')

Found 8016 images belonging to 2 classes.


In [5]:
validation_augmentation = ImageDataGenerator(
    rescale = 1./255
)

In [6]:
validation_generator = validation_augmentation.flow_from_directory(
    directory='./Testing',
    target_size=(128,128),
    batch_size=128,
    class_mode='binary'
)

Found 8016 images belonging to 2 classes.


In [7]:
# Getting pretrained model with imagenet weight by skipping top
conv_base = VGG16(input_shape=(128,128,3),include_top=False, weights= 'imagenet')

In [8]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [10]:
model=Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________


In [15]:
conv_base.trainable = True
set_trainable = False

for layer in conv_base.layers:
   if layer.name == 'block5_conv1':
     set_trainable = True
   if set_trainable:
     layer.trainable = True
   else:
     layer.trainable = False

In [16]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(train_generator,steps_per_epoch=8,
                  epochs=5,
                  verbose=1,
                  validation_data=validation_generator)

Epoch 1/5
8/8 [==============================] - 288s 39s/step - loss: 62.0408 - accuracy: 0.4980 - val_loss: 0.6923 - val_accuracy: 0.5031
Epoch 2/5
8/8 [==============================] - 242s 34s/step - loss: 0.6897 - accuracy: 0.5205 - val_loss: 0.6915 - val_accuracy: 0.5096
Epoch 3/5
8/8 [==============================] - 250s 35s/step - loss: 0.7000 - accuracy: 0.5117 - val_loss: 0.6900 - val_accuracy: 0.5226
Epoch 4/5
8/8 [==============================] - 251s 35s/step - loss: 0.6886 - accuracy: 0.5381 - val_loss: 0.6860 - val_accuracy: 0.5266
Epoch 5/5
8/8 [==============================] - ETA: 0s - loss: 0.6863 - accuracy: 0.5479

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'],loc='upper left')
plt.show()


In [ ]:

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'],loc='upper left')
plt.legend(['validation'],loc='upper left')

plt.show()